<a href="https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/integrations/chat/maritalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maritalk

MariTalk 是巴西公司 [Maritaca AI](https://www.maritaca.ai) 开发的助手。
MariTalk 基于经过特别训练、能很好地理解葡萄牙语的语言模型。

本 notebook 将通过两个示例演示如何使用 MariTalk 与 LangChain：

1. 一个使用 MariTalk 执行任务的简单示例。
2. LLM + RAG：第二个示例展示了如何回答一个问题，该问题的答案包含在一个不适合 MariTalk token 限制的长文档中。为此，我们将使用一个简单的搜索器 (BM25) 来首先在文档中搜索最相关的部分，然后将它们提供给 MariTalk 进行回答。

## 安装
首先，使用以下命令安装 LangChain 库（及其所有依赖项）：

In [ ]:
!pip install langchain langchain-core langchain-community httpx

## API 密钥

你需要一个 API 密钥，可以在 chat.maritaca.ai（“API 密钥”部分）获取。

### 示例 1 - 宠物名字建议

让我们定义语言模型 ChatMaritalk，并用您的 API 密钥进行配置。

In [ ]:
from langchain_community.chat_models import ChatMaritalk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate

llm = ChatMaritalk(
    model="sabia-2-medium",  # Available models: sabia-2-small and sabia-2-medium
    api_key="",  # Insert your API key here
    temperature=0.7,
    max_tokens=100,
)

output_parser = StrOutputParser()

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant specialized in suggesting pet names. Given the animal, you must suggest 4 names.",
        ),
        ("human", "I have a {animal}"),
    ]
)

chain = chat_prompt | llm | output_parser

response = chain.invoke({"animal": "dog"})
print(response)  # should answer something like "1. Max\n2. Bella\n3. Charlie\n4. Rocky"

### 流式生成

对于涉及长文本生成的任务，例如撰写一篇深度文章或翻译一份大型文档，接收分块生成的响应比等待完整文本的生成要更有优势。这使得应用程序在响应速度和效率上都有提升，特别是在生成文本量庞大的情况下。我们提供两种方法来满足这一需求：一种是同步方法，另一种是异步方法。

#### 同步：

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Suggest 3 names for my dog")]

for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)

#### 异步：

In [ ]:
from langchain_core.messages import HumanMessage


async def async_invoke_chain(animal: str):
    messages = [HumanMessage(content=f"Suggest 3 names for my {animal}")]
    async for chunk in llm._astream(messages):
        print(chunk.message.content, end="", flush=True)


await async_invoke_chain("dog")

### 示例 2 - RAG + LLM: UNICAMP 2024 入学考试问答系统
对于此示例，我们需要安装一些额外的库：

In [ ]:
!pip install unstructured rank_bm25 pdf2image pdfminer-six pikepdf pypdf unstructured_inference fastapi kaleido uvicorn "pillow<10.1.0" pillow_heif -q

#### 加载数据库

第一步是根据通知中的信息创建一个数据库。为此，我们将从 COMVEST 网站下载通知，并将提取的文本分割成 500 个字符的窗口。

In [ ]:
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Loading the COMVEST 2024 notice
loader = OnlinePDFLoader(
    "https://www.comvest.unicamp.br/wp-content/uploads/2023/10/31-2023-Dispoe-sobre-o-Vestibular-Unicamp-2024_com-retificacao.pdf"
)
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["\n", " ", ""]
)
texts = text_splitter.split_documents(data)

#### 创建搜索器
现在我们有了数据库，还需要一个搜索器。在本例中，我们将使用一个简单的 BM25 作为搜索系统，但这可以替换为任何其他搜索器（例如，通过嵌入进行搜索）。

In [ ]:
from langchain_community.retrievers import BM25Retriever

retriever = BM25Retriever.from_documents(texts)

#### 结合搜索系统 + LLM
现在我们有了搜索器，只需要实现一个指定任务的提示，然后调用链。

In [ ]:
from langchain.chains.question_answering import load_qa_chain

prompt = """Baseado nos seguintes documentos, responda a pergunta abaixo.

{context}

Pergunta: {query}
"""

qa_prompt = ChatPromptTemplate.from_messages([("human", prompt)])

chain = load_qa_chain(llm, chain_type="stuff", verbose=True, prompt=qa_prompt)

query = "Qual o tempo máximo para realização da prova?"

docs = retriever.invoke(query)

chain.invoke(
    {"input_documents": docs, "query": query}
)  # Should output something like: "O tempo máximo para realização da prova é de 5 horas."